<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


In [ ]:
### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests

# Define the static URL for the Falcon9 Launch HTML page
static_url =  "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the HTTP GET request
response = requests.get(static_url)

# Check the status code to ensure the request was successful
if response.status_code == 200:
    print("Request was successful!")
else:
    print(f"Request failed with status code: {response.status_code}")

Request was successful!


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
from bs4 import BeautifulSoup
import requests

# Define the static URL for the Falcon9 Launch HTML page
static_url ="https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922" 

# Perform the HTTP GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    # Create a BeautifulSoup object from the response content
    soup = BeautifulSoup(response.text, 'html.parser')
    print("BeautifulSoup object created successfully!")
else:
    print(f"Request failed with status code: {response.status_code}")

BeautifulSoup object created successfully!


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
from bs4 import BeautifulSoup
import requests

# Define the static URL for the Falcon9 Launch HTML page
static_url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'

# Perform the HTTP GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    # Create a BeautifulSoup object from the response content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Print the page title to verify the BeautifulSoup object was created properly
    print("Page Title:", soup.title.string)
else:
    print(f"Request failed with status code: {response.status_code}")

Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [10]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`

from bs4 import BeautifulSoup
import requests

# URL of the wiki page with the Falcon 9 launch data
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'

# Perform the HTTP GET request
response = requests.get(url)
html_content = response.text

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all tables on the page
html_tables = soup.find_all('table')

# Print the number of tables found
print(f"Number of tables found: {len(html_tables)}")

# Print the first table to inspect
print(html_tables[0])

Number of tables found: 15
<table class="wikitable plainrowheaders collapsible" id="2023ytd" style="width: 100%;">
<tbody><tr class="is-sticky">
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_22-0"><a href="#cite_note-booster-22"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch<br/>site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_23-0"><a href="#cite_note-Dragon-23"><span class="cite-bracket">[</span>d<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a h

Starting from the third table is our target table contains the actual launch records.


In [12]:
# Let's print the third table and check its content

from bs4 import BeautifulSoup
import requests

# URL of the wiki page with the Falcon 9 launch data
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'

# Perform the HTTP GET request
response = requests.get(url)
html_content = response.text

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all tables on the page
html_tables = soup.find_all('table')

# Access the third table (index 2)
target_table = html_tables[2]

# Define the provided function (replace this with the actual function you have)
def extract_column_from_header(header):
    # Example implementation (replace with actual function logic)
    return header.get_text(strip=True) if header else None

# Initialize an empty list for column names
column_names = []

# Find all <th> elements in the table header
header_th_elements = target_table.find_all('th')

# Iterate over each <th> element
for th in header_th_elements:
    column_name = extract_column_from_header(th)
    if column_name and len(column_name) > 0:
        column_names.append(column_name)

# Print the column names
print(column_names)

['Date and time (UTC)[376]', 'Version,booster[c]', 'Launch site', 'Payload[d]', 'Orbit', 'Customer']


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [13]:


# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

from bs4 import BeautifulSoup
import requests

# URL of the wiki page with the Falcon 9 launch data
url ='https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'

# Perform the HTTP GET request
response = requests.get(url)
html_content = response.text

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all tables on the page
html_tables = soup.find_all('table')

# Assuming the first table is the relevant one
first_launch_table = html_tables[0]

# Define the provided function (replace this with the actual function you have)
def extract_column_from_header(header):
    # Example implementation (replace with actual function logic)
    return header.get_text(strip=True) if header else None

# Initialize an empty list for column names
column_names = []

# Find all <th> elements in the table header
header_th_elements = first_launch_table.find_all('th')

# Iterate over each <th> element
for th in header_th_elements:
    column_name = extract_column_from_header(th)
    if column_name and len(column_name) > 0:
        column_names.append(column_name)

# Print the column names
print(column_names)


['Flight No.', 'Date andtime (UTC)', 'Version,booster[c]', 'Launchsite', 'Payload[d]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '195', '196', 'FH 5', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', 'FH 6', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', 'FH 7', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', 'FH 8', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', 'FH 9', '285']


Check the extracted column names


In [14]:
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,booster[c]', 'Launchsite', 'Payload[d]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '195', '196', 'FH 5', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', 'FH 6', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', 'FH 7', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', 'FH 8', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', 'FH 9', '285']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [16]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# URL of the wiki page with the Falcon 9 launch data
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'

# Perform the HTTP GET request
response = requests.get(url)
html_content = response.text

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all tables on the page
html_tables = soup.find_all('table')

# Access the third table (index 2)
target_table = html_tables[2]

# Define the provided function (replace this with the actual function you have)
def extract_column_from_header(header):
    # Example implementation (replace with actual function logic)
    return header.get_text(strip=True) if header else None

# Initialize an empty list for column names
column_names = []

# Find all <th> elements in the table header
header_th_elements = target_table.find_all('th')

# Iterate over each <th> element
for th in header_th_elements:
    column_name = extract_column_from_header(th)
    if column_name and len(column_name) > 0:
        column_names.append(column_name)

# Create an empty dictionary with column names as keys
data_dict = {col: [] for col in column_names}

# Convert the dictionary into a Pandas DataFrame
df = pd.DataFrame(data_dict)

# Print the DataFrame to verify
print(df)

Empty DataFrame
Columns: [Date and time (UTC)[376], Version,booster[c], Launch site, Payload[d], Orbit, Customer]
Index: []


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [18]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re  # Regular expression module for cleaning data

# URL of the wiki page with the Falcon 9 launch data
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'

# Perform the HTTP GET request
response = requests.get(url)
html_content = response.text

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all tables on the page
html_tables = soup.find_all('table')

# Access the third table (index 2)
target_table = html_tables[2]

# Define the provided function (replace this with the actual function you have)
def extract_column_from_header(header):
    return header.get_text(strip=True) if header else None

# Initialize an empty list for column names
column_names = []

# Find all <th> elements in the table header
header_th_elements = target_table.find_all('th')

# Iterate over each <th> element
for th in header_th_elements:
    column_name = extract_column_from_header(th)
    if column_name and len(column_name) > 0:
        column_names.append(column_name)

# Create an empty dictionary with column names as keys
launch_dict = {col: [] for col in column_names}

# Function to clean data (remove references, annotations, etc.)
def clean_data(text):
    if text:
        # Remove reference annotations (e.g., [8])
        text = re.sub(r'\[\d+\]', '', text).strip()
        # Handle missing values (e.g., "N/A")
        if text == 'N/A':
            return None
    return text

# Extract rows from the table
rows = target_table.find_all('tr')

# Assuming the first row is the header, start from the second row
for row in rows[1:]:
    cells = row.find_all('td')
    
    # Ensure the number of cells matches the number of column names
    if len(cells) == len(column_names):
        for i, cell in enumerate(cells):
            column_name = column_names[i]
            cell_text = clean_data(cell.get_text())
            launch_dict[column_name].append(cell_text)

# Convert the dictionary into a Pandas DataFrame
df = pd.DataFrame(launch_dict)

# Print the DataFrame to verify
print(df)

   Date and time (UTC)[376]           Version,booster[c]  \
0                       TBD                 F9 B5B1083.4   
1    4 September 2024~12:59                        F9 B5   
2    5 September 2024~01:28                        F9 B5   
3          9 September 2024                        F9 B5   
4         13 September 2024                        F9 B5   
5   24 September 2024~18:30                F9 B5 B1085.2   
6            September 2024                        F9 B5   
7            September 2024                        F9 B5   
8                      2024                        F9 B5   
9                   Q3 2024                        F9 B5   
10           7 October 2024                        F9 B5   
11          10 October 2024  Falcon Heavy B5B1090 (core)   
12         Mid-October 2024                        F9 B5   
13             October 2024                        F9 B5   
14             October 2024                        F9 B5   
15             October 2024             

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [21]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re  # Regular expression module for cleaning data

# URL of the wiki page with the Falcon 9 launch data
url = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'

# Perform the HTTP GET request
response = requests.get(url)
html_content = response.text

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all tables on the page
html_tables = soup.find_all('table')

# Access the third table (index 2)
target_table = html_tables[2]

# Define the provided function (replace this with the actual function you have)
def extract_column_from_header(header):
    return header.get_text(strip=True) if header else None

# Initialize an empty list for column names
column_names = []

# Find all <th> elements in the table header
header_th_elements = target_table.find_all('th')

# Iterate over each <th> element
for th in header_th_elements:
    column_name = extract_column_from_header(th)
    if column_name and len(column_name) > 0:
        column_names.append(column_name)

# Create an empty dictionary with column names as keys
launch_dict = {col: [] for col in column_names}

# Function to clean data (remove references, annotations, etc.)
def clean_data(text):
    if text:
        # Remove reference annotations (e.g., [8])
        text = re.sub(r'\[\d+\]', '', text).strip()
        # Handle missing values (e.g., "N/A")
        if text == 'N/A':
            return None
    return text

# Extract rows from the table
rows = target_table.find_all('tr')

# Assuming the first row is the header, start from the second row
for row in rows[1:]:
    cells = row.find_all('td')
    
    # Ensure the number of cells matches the number of column names
    if len(cells) == len(column_names):
        for i, cell in enumerate(cells):
            column_name = column_names[i]
            cell_text = clean_data(cell.get_text())
            launch_dict[column_name].append(cell_text)

# Create the DataFrame using the provided method
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

# Print the DataFrame to verify
print(df)

   Date and time (UTC)[376]           Version,booster[c]  \
0                       TBD                 F9 B5B1083.4   
1    4 September 2024~12:59                        F9 B5   
2    5 September 2024~01:28                        F9 B5   
3          9 September 2024                        F9 B5   
4         13 September 2024                        F9 B5   
5   24 September 2024~18:30                F9 B5 B1085.2   
6            September 2024                        F9 B5   
7            September 2024                        F9 B5   
8                      2024                        F9 B5   
9                   Q3 2024                        F9 B5   
10           7 October 2024                        F9 B5   
11          10 October 2024  Falcon Heavy B5B1090 (core)   
12         Mid-October 2024                        F9 B5   
13             October 2024                        F9 B5   
14             October 2024                        F9 B5   
15             October 2024             

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
